# Data EDA
* 세줄 요약
  * Clustering 성능 생각보다 떨어짐
    * 1400/1600개 정도만 탐지됨.
    * 다양한 방법으로 시도했으나... 강제로 늘리면 오염된거랑 안된거랑 섞임  -> 포기
  * 직접 구현
    * 결론
      ```python
      (df['ascii_ratio'] >= 0.15) & 
      (df['special_char_ratio'] >= 0.030) &
      (df['english_count'] >= 2) 
      ```
    * 이렇게 하면 1543개가 추출되고 이중 2개만 잘못되었음
    * 1541/1600, 2/1543 정도로 나쁘지 않게 추출됨.

# Library import 단계

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Noise vs Non - Noise 분류
* Noise가 포함된 데이터가 1600개
  * 이때 Noise는 기사의 제목의 일부는 랜던함 `ASCII`로 변환함.
* Non - Noise가 포함된 데이터가 1200개
  * Non - Noise에서 제대로 Label된 데이터가 200개, label의 섞인 데이터가 1000개

In [35]:
# train_data 불러오기
data = pd.read_csv('./data/train.csv')
data.head()

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6


In [36]:
## 데이터 분포 확인.
label_distribution = []
for num in range(7):
    cond = data['target'] == num
    result = len(data[cond])
    label_distribution.append(result)
label_distribution


[397, 410, 388, 385, 406, 419, 395]

In [37]:
def mapping_label(df):
    # 딕셔너리로 매핑 정의
    label_map = {
        0:'생활문화',1:'스포츠',2:'정치',3:'사회',4:'IT과학',5:'경제',6:'세계'
    }
    df['target'] = df['target'].map(label_map)
    return df

df = mapping_label(data)
df.head(3)

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,IT과학
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,사회
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",정치


## 아스키, 영어 및 대문자 관련 비율 확인 진행
* 아스키 비율
* 아스키 중 영어 비율
* 영어 중 대문자 비율
* 아스키 중 대문자 비율

In [38]:
## 현재 문장에서 ascii의 비율과, ascii의 비율 중 영어와 대문자가 있는 경우에 대한 확인.
def calculate_ascii(text):
    # 공백을 제외한 ASCII 문자들
    ascii_chars = [char for char in text if ord(char) < 128 and not char.isspace()]
    ascii_count = len(ascii_chars)
    ascii_ratio = ascii_count / len(text)

    # ASCII 문자들 중 영어 문자만 추출
    english_chars = [char for char in ascii_chars if char.isalpha()]
    has_english = len(english_chars) > 0
    
    # 영어 관련 체크
    is_english_only = all(char.isalpha() for char in ascii_chars) if has_english else False
    
    # 대문자 관련 계산
    is_all_uppercase = all(char.isupper() for char in english_chars) if has_english else False
    
    # ASCII 문자 중 대문자 비율
    ascii_uppercase_count = sum(1 for char in ascii_chars if char.isupper())
    ascii_uppercase_ratio = ascii_uppercase_count / len(ascii_chars) if ascii_chars else 0
    
    # 영어 문자 중 대문자 비율
    english_uppercase_count = sum(1 for char in english_chars if char.isupper())
    english_uppercase_ratio = english_uppercase_count / len(english_chars) if english_chars else 0
    
    return ascii_count, is_english_only, is_all_uppercase, ascii_ratio, ascii_uppercase_ratio, english_uppercase_ratio

df['text'] = df['text'].str.replace('…', ' ', regex=False)
df['text'] = df['text'].str.replace('...', ' ', regex=False)
df['text'] = df['text'].str.replace('·', ' ', regex=False)
df['length'] = df['text'].map(len)
df['ascii_count'], df['is_english_only'], df['is_all_uppercase'], df['ascii_ratio'], df['ascii_uppercase_ratio'], df['english_uppercase_ratio'] = zip(*df['text'].apply(calculate_ascii))

# 결과 확인
df[['text', 'ascii_count', 'is_english_only', 'is_all_uppercase', 'ascii_ratio', 'ascii_uppercase_ratio', 'english_uppercase_ratio']].head()

,text,ascii_count,is_english_only,is_all_uppercase,ascii_ratio,ascii_uppercase_ratio,english_uppercase_ratio
0,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,13,False,False,0.406250,0.307692,0.571429
1,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,16,False,False,0.592593,0.375000,0.750000
2,"m 김정) 자주통일 새,?r열1나가야1보",7,False,False,0.318182,0.000000,0.000000
3,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,3,False,False,0.103448,0.000000,0.000000
4,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,9,False,False,0.321429,0.333333,0.500000


## 서현이 처리 추가
* 특수 기호의 개수 세는 함수 사용
* 특수 기호의 개수 및 전체 길이중 특수 기호의 비율 확인


In [39]:
special_char_pattern = r'(?<!\d)\.(?!\d)|(?<!\d)%|[^가-힣A-Z\u4E00-\u9FFF\s0-9\.%㎜㎡]'

# 각 text에 포함된 특수 기호의 개수를 세는 함수 정의
def count_special_characters(text):
    return len(re.findall(special_char_pattern, text))

# 데이터프레임에 새로운 열 추가
df['special_char_count'] = df['text'].apply(count_special_characters)
df['special_char_ratio'] = df['special_char_count'] / df['text'].str.len()
df.head(2)

,ID,text,target,length,ascii_count,is_english_only,is_all_uppercase,ascii_ratio,ascii_uppercase_ratio,english_uppercase_ratio,special_char_count,special_char_ratio
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,IT과학,32,13,False,False,0.406250,0.307692,0.571429,6,0.187500
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,사회,27,16,False,False,0.592593,0.375000,0.750000,7,0.259259


## 채은이 과정 추가
* r가d나d다 3
* r가dd다 2
* rdd다 1

In [40]:
df['english_count'] = df['text'].str.findall(r'[A-Za-z]+').str.len()
df.head(1)

,ID,text,target,length,ascii_count,is_english_only,is_all_uppercase,ascii_ratio,ascii_uppercase_ratio,english_uppercase_ratio,special_char_count,special_char_ratio,english_count
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,IT과학,32,13,False,False,0.40625,0.307692,0.571429,6,0.1875,6


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2800 entries, 0 to 2799
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       2800 non-null   object 
 1   text                     2800 non-null   object 
 2   target                   2800 non-null   object 
 3   length                   2800 non-null   int64  
 4   ascii_count              2800 non-null   int64  
 5   is_english_only          2800 non-null   bool   
 6   is_all_uppercase         2800 non-null   bool   
 7   ascii_ratio              2800 non-null   float64
 8   ascii_uppercase_ratio    2800 non-null   float64
 9   english_uppercase_ratio  2800 non-null   float64
 10  special_char_count       2800 non-null   int64  
 11  special_char_ratio       2800 non-null   float64
 12  english_count            2800 non-null   int64  
dtypes: bool(2), float64(4), int64(4), object(3)
memory usage: 246.2+ KB


## Clustering 진행

In [10]:
df.columns

Index(['ID', 'text', 'target', 'length', 'ascii_count', 'is_english_only',
       'is_all_uppercase', 'ascii_ratio', 'ascii_uppercase_ratio',
       'english_uppercase_ratio', 'special_char_count', 'special_char_ratio',
       'english_count'],
      dtype='object')

In [ ]:
# 1. 특성 구분
binary_features = ['is_english_ascii', 'is_all_uppercase', 'is_english_only']
numeric_features = ['ascii_count', 'ascii_ratio', 'uppercase_ratio', 
                   'ascii_uppercase_ratio', 'english_uppercase_ratio', 
                   'special_char_count', 'special_char_ratio', 'english_count']


# 2. 데이터 전처리
X_binary = df[binary_features].astype(int)
scaler = StandardScaler()
X_numeric = pd.DataFrame(scaler.fit_transform(df[numeric_features]), 
                        columns=numeric_features)
X = pd.concat([X_numeric, X_binary], axis=1)

# 2. 최적의 eps 값 찾기
nearest_neighbors = NearestNeighbors(n_neighbors=2)
neighbors = nearest_neighbors.fit(X)
distances, indices = neighbors.kneighbors(X)
distances = np.sort(distances[:, 1])

plt.figure(figsize=(10, 5))
plt.plot(range(len(distances)), distances)
plt.title('K-distance Graph')
plt.xlabel('Data Points sorted by distance')
plt.ylabel('Epsilon')
plt.show()
dbscan = DBSCAN(eps=0.649728, min_samples=2)  # eps 값은 그래프를 보고 조정할 수 있습니다
df['predicted_binary'] = dbscan.fit_predict(X)
if -1 in df['predicted_binary'].unique():
    for idx in df[df['predicted_binary'] == -1].index:
        # 가장 가까운 non-noise 포인트의 클러스터로 할당
        non_noise_mask = df['predicted_binary'] != -1
        if non_noise_mask.any():
            distances = np.sqrt(((X.loc[idx] - X[non_noise_mask]) ** 2).sum(axis=1))
            nearest_cluster = df['predicted_binary'][non_noise_mask].iloc[distances.argmin()]
            df.loc[idx, 'predicted_binary'] = nearest_cluster

# 4. 클러스터 레이블 조정 (0, 1로 매핑)
unique_labels = df['predicted_binary'].unique()
if len(unique_labels) > 2:
    # 2개 이상의 클러스터가 생긴 경우, 가장 큰 2개의 클러스터만 사용
    value_counts = df['predicted_binary'].value_counts()
    main_clusters = value_counts.nlargest(2).index
    df['predicted_binary'] = df['predicted_binary'].map(
        {main_clusters[0]: 0, main_clusters[1]: 1}
    )
# 4. 결과 확인 
#df['predicted_binary'] = df['predicted_binary'].fillna(1).astype(int)
df['predicted_binary'].value_counts()

In [ ]:
cond = df['predicted_binary'] == 0
texts_with_1 = df[cond]['text']
# 랜덤하게 200개 추출 후 사용
sampled_texts = texts_with_1.sample(n=min(200, len(texts_with_1)), random_state=42)
for idx, text in enumerate(sampled_texts, 1):
    print(f"{idx:3d}. Length: {len(text):4d} | Text: {text[:200]}{'...' if len(text) > 200 else ''}")
    if idx % 10 == 0:  # 10개마다 구분선 추가
        print('-' * 100)

In [ ]:
numeric_features = ['ascii_count', 'ascii_ratio', 'uppercase_ratio', 
                   'ascii_uppercase_ratio', 'english_uppercase_ratio', 
                   'special_char_count', 'special_char_ratio', 'english_count']


# 2. 데이터 전처리
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(df[numeric_features]), 
                        columns=numeric_features)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X)
df['clusters_results'] = clusters
print(df['clusters_results'].value_counts())

In [ ]:
cond = df['clusters_results'] == 2
texts_with_1 = df[cond]['text']
# 랜덤하게 200개 추출 후 사용
sampled_texts = texts_with_1.sample(n=min(200, len(texts_with_1)), random_state=42)
for idx, text in enumerate(sampled_texts, 1):
    print(f"{idx:3d}. Length: {len(text):4d} | Text: {text[:200]}{'...' if len(text) > 200 else ''}")
    if idx % 10 == 0:  # 10개마다 구분선 추가
        print('-' * 100)

## Clustering  포기
* 1400개 정도에 대한 구분은 완벽하지만 200개에 대한 탐지가 거의 불가능
* 순수 손으로 진행.

In [ ]:
features = ['ascii_count', 'ascii_ratio', 'uppercase_ratio', 
            'ascii_uppercase_ratio', 'english_uppercase_ratio', 
            'special_char_count', 'special_char_ratio', 'english_count']

In [60]:
final_conditions = (
    (df['ascii_ratio'] >= 0.15) & 
    (df['special_char_ratio'] >= 0.030) &
    (df['english_count'] >= 2) 
    #(df['ascii_uppercase_ratio'] <= 0.8) 
)

df['result'] = final_conditions
print(f"\nSelected records: {len(df[final_conditions])}")



Selected records: 1543


In [ ]:
cond = df['result'] == True
texts_with_1 = df[cond]['text']
# 랜덤하게 200개 추출 후 출력
sampled_texts = texts_with_1.sample(n=min(200, len(texts_with_1)), random_state=42)
for idx, text in enumerate(sampled_texts, 1):
    print(f"Text: {text[:200]}{'...' if len(text) > 200 else ''}")
    if idx % 10 == 0:  # 10개마다 구분선 추가
        print('-' * 100)

In [54]:
cond = df['result'] == True
df_ascii = df[cond]
len(df_ascii), df_ascii.columns

(1543,
 Index(['ID', 'text', 'target', 'length', 'ascii_count', 'is_english_only',
        'is_all_uppercase', 'ascii_ratio', 'ascii_uppercase_ratio',
        'english_uppercase_ratio', 'special_char_count', 'special_char_ratio',
        'english_count', 'result'],
       dtype='object'))

In [55]:
df_ascii = df_ascii[['ID', 'text', 'target']]

In [56]:
df_ascii.to_csv('ascii.csv')